In [44]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
import frechet_video_distance as fvd
import numpy as np
# Number of videos must be divisible by 16.
NUMBER_OF_VIDEOS = 16
VIDEO_LENGTH = 15


In [45]:
vid1 = np.stack([np.load("./independent_10_7/sample_0000-0.npy"),
                 np.load("./independent_10_7/sample_0000-3.npy")])
vid2 = np.stack([np.load("./independent_10_7/sample_0001-0.npy"),
                 np.load("./independent_10_7/sample_0002-0.npy")])


# # hack to have number of videos be divisible by 16 
vid1 = np.repeat(vid1, 8, 0)
vid2 = np.repeat(vid2, 8, 0)

# reshape to have channel in last dim
vid1 = np.moveaxis(vid1, 2, 4) # B, T, H, W, C
vid2 = np.moveaxis(vid2, 2, 4) # B, T, H, W, C

In [50]:
def compute_FVD_features(vid1, vid2): 
    # batch has to be divisible by 16 in create_id3_embedding
    assert vid1.shape[0] % 16 == 0 
    assert vid2.shape[0] % 16 == 0

    # assert channel is last dimension
    # and trust that shape is B, T, H, W, C
    assert vid1.shape[4] == 3
    assert vid2.shape[4] == 3
    
    with tf.Graph().as_default():
        vid1 = tf.convert_to_tensor(vid1, np.uint8)
        vid2 = tf.convert_to_tensor(vid2, np.uint8)
        
        vid1_feature_vec = fvd.create_id3_embedding(fvd.preprocess(vid1,(224, 224)))
        vid2_feature_vec = fvd.create_id3_embedding(fvd.preprocess(vid2,(224, 224)))
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.tables_initializer())
            return sess.run(vid1_feature_vec), sess.run(vid2_feature_vec)


In [47]:
def compute_FVD(vid1_features, vid2_features): 
    # no batch limitations here, can pass in entire dataset
    with tf.Graph().as_default():
        vid1_features = tf.convert_to_tensor(vid1_features, np.float32)
        vid2_features = tf.convert_to_tensor(vid2_features, np.float32)
        
        result = fvd.calculate_fvd(vid1_features, vid2_features)

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.tables_initializer())
            return sess.run(result)


In [48]:
%%time
vid1_feature_vec, vid2_feature_vec = compute_FVD_features(vid1[:,:15,:,:,:], vid2[:,:15,:,:,:])

2022-04-21 16:38:54.965856: W tensorflow/core/common_runtime/graph_constructor.cc:1511] Importing a graph with a lower producer version 26 into an existing graph with producer version 987. Shape inference will have run different parts of the graph with different producer versions.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


CPU times: user 1min 20s, sys: 2.13 s, total: 1min 22s
Wall time: 45.2 s


In [49]:
result = compute_FVD(vid1_feature_vec, vid2_feature_vec)
result

1055.9459